In [24]:
import re
import sys
import warnings
import nltk
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt_tab')

# Adjust this path to where your NLTK data is located
nltk.data.path.append('C:\\Users\\Tobias E\\AppData\\Roaming\\nltk_data')

# Suppress all warning messages
if not sys.warnoptions:
    warnings.simplefilter("ignore")



[nltk_data] Downloading package punkt_tab to C:\Users\Tobias
[nltk_data]     E\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


In [25]:
# 1. Load the data
data_path = "C:\\workspace\\book1.csv"
data_raw = pd.read_csv(data_path, encoding='latin1')

In [26]:
# 2. Shuffle the data
data_raw = data_raw.sample(frac=1)

In [27]:
# 3. Identify category columns
categories = list(data_raw.columns.values)
categories = categories[2:]  # Adjust if your CSV structure changes

In [28]:
# 4. Basic cleaning on the "Heading" column
data_raw['Heading'] = (
    data_raw['Heading']
    .str.lower()
    .str.replace('[^\w\s]', '', regex=True)     # remove punctuation
    .str.replace('\d+', '', regex=True)         # remove digits
    .str.replace('<.*?>', '', regex=True)       # remove HTML tags
)

In [29]:
# 5. Download stopwords and define your stop-word removal function
nltk.download('stopwords')
stop_words = set(stopwords.words('swedish'))

def removeStopWords(sentence):
    return " ".join(
        [word for word in nltk.word_tokenize(sentence) 
         if word not in stop_words]
    )

data_raw['Heading'] = data_raw['Heading'].apply(removeStopWords)

[nltk_data] Downloading package stopwords to C:\Users\Tobias
[nltk_data]     E\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [30]:
# 6. (Optional) Stemming
stemmer = SnowballStemmer("swedish")

def stemming(sentence):
    stemSentence = ""
    for word in sentence.split():
        stemSentence += stemmer.stem(word) + " "
    return stemSentence.strip()

# If you want to apply stemming, uncomment:
# data_raw['Heading'] = data_raw['Heading'].apply(stemming)

In [31]:
# 7. Split the data
train, test = train_test_split(data_raw, random_state=42, test_size=0.30, shuffle=True)

train_text = train['Heading']
test_text = test['Heading']

In [32]:
# 8. Vectorize using TF-IDF
vectorizer = TfidfVectorizer(strip_accents='unicode', 
                             analyzer='word', 
                             ngram_range=(1,3), 
                             norm='l2')
vectorizer.fit(train_text)

x_train = vectorizer.transform(train_text)
y_train = train.drop(labels=['Id', 'Heading'], axis=1)

x_test = vectorizer.transform(test_text)
y_test = test.drop(labels=['Id', 'Heading'], axis=1)

In [33]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()

In [ ]:
# Fit the model for each category separately
for category in y_train.columns:
	clf.fit(x_train, y_train[category])
#	print(f"Fitted model for category: {category}")

Fitted model for category: Politik
Fitted model for category: Utbildning
Fitted model for category: Religion
Fitted model for category: Miljo
Fitted model for category: Ekonomi
Fitted model for category: LivsstilFritt
Fitted model for category: SamhalleKonflikter
Fitted model for category: Halsa
Fitted model for category: Idrott
Fitted model for category: VetenskapTeknik


In [36]:
y_pred = clf.predict(x_test)

In [41]:
from sklearn.metrics import accuracy_score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and binary targets